In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200411132612-0005
KERNEL_ID = 83c6a9d9-c410-46b9-bf64-9863607ac113
Libraries imported.


In [2]:
#Reading in the table and setting the header to the first row
df = pd.read_html(r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)
df = pd.DataFrame(df[0])
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [3]:
cdf= df[df['Borough']!='Not assigned']
cdf.reset_index(inplace = True, drop = True)

In [4]:
cdf_fil= cdf[cdf['Borough'].str.contains('Toronto')]
cdf_fil.head(20)

,Postal code,Borough,Neighborhood
2,M5A,Downtown Toronto,Regent Park / Harbourfront
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
9,M5B,Downtown Toronto,"Garden District, Ryerson"
15,M5C,Downtown Toronto,St. James Town
19,M4E,East Toronto,The Beaches
20,M5E,Downtown Toronto,Berczy Park
24,M5G,Downtown Toronto,Central Bay Street
25,M6G,Downtown Toronto,Christie
30,M5H,Downtown Toronto,Richmond / Adelaide / King
31,M6H,West Toronto,Dufferin / Dovercourt Village


In [5]:
#Replacing the "/" with a ","
cdf_fil['Neighborhood'] = cdf_fil['Neighborhood'].replace("/", ",", regex = True)
cdf_fil.head()

/opt/ibm/conda/miniconda3.6/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,Postal code,Borough,Neighborhood
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
9,M5B,Downtown Toronto,"Garden District, Ryerson"
15,M5C,Downtown Toronto,St. James Town
19,M4E,East Toronto,The Beaches


In [6]:
for i in cdf_fil['Neighborhood']:
    if i=='NaN':
      print (i)

In [7]:
cdf_fil.shape

(39, 3)

In [8]:
## Importing the CSV file with geo-loc data
geo = pd.read_csv(r'http://cocl.us/Geospatial_data', header = 0)
geo.shape

(103, 3)

In [9]:
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
pdf= pd.DataFrame(columns= ['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'])

In [11]:
for i in cdf['Postal code']:
    for j in geo['Postal Code']:
        if i==j:
               pdf['Postal Code']=cdf['Postal code']
               pdf['Borough']=cdf['Borough']
               pdf['Neighborhood']=cdf['Neighborhood']
               pdf['Latitude']= geo['Latitude']
               pdf['Longitude']=geo['Longitude']

In [12]:
pdf= pdf[pdf['Borough'].str.contains('Toronto')]
pdf.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.763573,-79.188711
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.773136,-79.239476
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
15,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
19,M4E,East Toronto,The Beaches,43.786947,-79.385975
